In [ ]:
import sys
sys.path.append('../../../config.py')
from config import YOUR_NAME

pip install ndjson

In [ ]:
PROJECT_ID = "training1emakia"  # @param {type:"string"}
print(PROJECT_ID)

In [ ]:
BUCKET_NAME = f"{YOUR_NAME}_batch_prediction"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
print(BUCKET_NAME)
print(BUCKET_URI)

In [ ]:
#create the bucket 
!gsutil mb -b on -l us-central1 gs://{YOUR_NAME}_batch_prediction

In [ ]:
import pandas as pd


gcs_source = f'gs://{YOUR_NAME}_batch_prediction2/dataforbatchprediction_csv.csv'
data = pd.read_csv(
    gcs_source,      # relative python path to subdirectory
    sep=',',           # Tab-separated value file.
    #quotechar="'",        # single quote allowed as quote character
    usecols=['text']  # Only load the three columns specified.
    #na_values=['.', '??']       # Take any '.' or '??' values as NA
)
print(data.keys())
print(data.text[0])
#print(data[1])

In [21]:
# Instantiate the Storage client and create the new bucket
from google.cloud import  storage

input_file_name = "batch-prediction-input.jsonl"

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, t in enumerate(data.text):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(t)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)